# MLA 703. RNN-LSTM et architectures avancées [Analyse de sentiment]

In [ ]:
# Dans ce notebook, nous allons nous intéresser à des tâches d'analyse de sentiments
# -> c'est à dire prédire un label de sentiment (ici positif ou négatif) à partir d'un texte

# Ce notebook vise à approfondir : 
# - L'application du DL sur des données textuelles
# - La compréhension des architectures RNN avancées comme les LSTM et les mécanismes d'attention
#

# Importation des modules

In [1]:
# On importe les librairies usuelless
import math
import numpy as np
import matplotlib.pyplot as plt

# On désactive les warnings
import warnings
warnings.filterwarnings('ignore')


# Fonction utile 

In [2]:
def plot_history(history):
  # On trace la loss et l'accuracy du modèle
  # On trace l'évolution de l'accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy']) 
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # On trace l'évolution de la loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [3]:
def predict(model):

  # On prédit sur l'ensemble de test

  # On prédit sur les données de test
  y_hat = model.predict(x_test)

  # On tranforme les prédictions en labels
  i_pos = [i for i in range(len(y_hat)) if y_hat[i]>0]
  i_neg = [i for i in range(len(y_hat)) if y_hat[i]<=0]

  y_pred   = np.zeros(len(y_hat))
  y_pred[i_pos] = 1
  y_pred[i_neg] = 0
  return y_pred

## 1. Charger les données

In [4]:
# On charge directement la base IMDB par les datasets de Keras
# La méthode load_date possède pas mal d'options et de propriétés intéressantes (voir description) :
# 1) Sépare les jeux d'entrainement et de test
# 2) num_words : Top most frequent words to consider.
# 3) skip_top : Top most frequent words to ignore (they will appear as oov_char value in the sequence data).
# 4) maxlen : Maximum sequence length. Any longer sequence will be truncated.
# 5) seed : Seed for reproducible data shuffling.
# 6) start_char : The start of a sequence will be marked with this character. Set to 1 because 0 is usually the padding character.
# 7) oov_char : words that were cut out because of the num_words or skip_top limit will be replaced with this character.
# 8) index_from : Index actual words with this index and higher.

from tensorflow.keras.datasets import imdb

num_words = 5000
max_len   =  100

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words,
                                                      maxlen=max_len)
                                            

2023-02-28 14:51:00.978271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 14:51:00.978288: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


17464789/17464789 [==============================] - 1s 0us/step


In [8]:
x_train

array([list([1, 778, 128, 74, 12, 630, 163, 15, 4, 1766, 2, 1051, 2, 32, 85, 156, 45, 40, 148, 139, 121, 664, 665, 10, 10, 1361, 173, 4, 749, 2, 16, 3804, 8, 4, 226, 65, 12, 43, 127, 24, 2, 10, 10]),
       list([1, 54, 13, 1610, 14, 20, 13, 69, 55, 364, 1398, 21, 54, 13, 219, 12, 13, 1706, 15, 4, 20, 16, 329, 6, 176, 329, 74, 51, 13, 873, 4, 156, 71, 78, 4, 2, 322, 16, 31, 7, 4, 249, 4, 65, 16, 38, 379, 12, 100, 157, 18, 6, 910, 20, 549, 18, 4, 1496, 21, 14, 31, 9, 24, 6, 212, 12, 9, 6, 1322, 991, 7, 3002, 4, 425, 9, 73, 2218, 549, 18, 31, 155, 36, 100, 763, 379, 20, 103, 351, 2, 13, 202, 12, 2241, 5, 6, 320, 46, 7, 457]),
       list([1, 4, 204, 2, 20, 16, 93, 11, 2, 19, 2, 4390, 6, 55, 52, 22, 849, 4227, 119, 7, 2, 961, 178, 6, 1018, 221, 20, 1184, 2, 2, 29, 7, 265, 16, 530, 17, 29, 220, 210, 468, 8, 30, 11, 32, 7, 27, 102, 2, 3634, 17, 3278, 1881, 16, 6, 2, 7, 1262, 190, 4, 20, 122, 2353, 8, 79, 6, 117, 196, 11, 1370, 12, 127, 24, 847, 33, 4, 1062, 7, 4, 2, 310, 131, 12, 9, 6, 253,

In [10]:
y_train, y_train.shape

(array([0, 0, 1, ..., 0, 1, 1]), (2773,))

## 2. Formater/Préparer les données

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# On padde les séquences

print("La phrase avant padding est : \n {}". format(x_train[0]))

# On padde les séquence de mot
max_len     = 100
x_train       =  pad_sequences(x_train, maxlen=max_len, truncating='post')
x_test        = pad_sequences(x_test, maxlen=max_len, truncating='post')

print("La phrase paddée sur une longueur {} est : \n {}". format(max_len, x_train[0]))
      
print(len(x_train))    

La phrase avant padding est : 
 [1, 778, 128, 74, 12, 630, 163, 15, 4, 1766, 2, 1051, 2, 32, 85, 156, 45, 40, 148, 139, 121, 664, 665, 10, 10, 1361, 173, 4, 749, 2, 16, 3804, 8, 4, 226, 65, 12, 43, 127, 24, 2, 10, 10]
La phrase paddée sur une longueur 100 est : 
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    1  778  128   74   12  630  163   15    4 1766    2 1051    2
   32   85  156   45   40  148  139  121  664  665   10   10 1361  173
    4  749    2   16 3804    8    4  226   65   12   43  127   24    2
   10   10]
2773


## 3. Déclaration du réseau

La tâche consiste à prédire la valence d'un texte à partir de son contenu.
Pour ce faire nous allons créer une architecture many-to-one 
avec les réseaux RNNs
Le problème consiste à implémenter et apprendre
des réseaux avec - par exemple - les configurations suivantes : 
On va comparer les configurations suivantes : 
- Simple RNN avec dropout (avec un taux de 25% en sortie du RNN)
- RNN-LSTM gauche-droite classique
- LSTM bi-directionnel et return_sequences = False
- Plusieurs couches au choix. On veillera en particulier 
  à la valeur des arguments return_sequences
  
Dans le rapport à faire directement dans le notebook, on prendra soin de :
- reporter les losses sur les ensembles d'entrainement et de validation
- mesurer l'accuracy sur l'ensemble de test
On commentera les résultats obtenus en comparant les configurations
Quelle est la configuration donnant la meilleure performance ? 
Pourquoi ?

Aide : une couche LSTM bi-directionnelle est obtenue en appliquant sur la même couche une couche LSTM et une couche Bidirectional

# Simple RNN avec dropout (avec un taux de 25% en sortie du RNN)

In [ ]:
# On importe les librairies pour le RNN
from tensorflow.keras.layers import Dense , Input , SimpleRNN, LSTM , Embedding, Dropout
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D,MaxPooling1D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import  Adam
#from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 128                                                   # dimension de l'embedding
RNN_size   = 64

# Example d'architecture pour le réseau RNN simple avec dropout
model_rnn_simple = Sequential()
model_rnn_simple.add(Embedding(num_words, embed_size))                 # layer embedding
model_rnn_simple.add(SimpleRNN(RNN_size, return_sequences = False))    # layer RNN
model_rnn_simple.add(Dropout(0.25))                                    # layer Dropout
model_rnn_simple.add(Dense(1))                                         # layer Dense

# On affiche l'architecture de notre modèle
model_rnn_simple.summary()

# On spécifie la fonction de perte, l'optimiseur, et la fonction d'évaluation
model_rnn_simple.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## 4. Entrainement du réseau

In [ ]:
# On entraine le réseau
batch_size = 64                                                             # tailles des mini-batch
epochs = 10                                                                 # nombre d'époques
history_rnn_simple = model_rnn_simple.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2) # on entraine

In [ ]:
plot_history(history_rnn_simple)

In [ ]:
y_pred=predict(model_rnn_simple)

In [ ]:
# On importe les librairies pour l'évaluation
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

# On calcule la matrice de confusion
cm_test = confusion_matrix(y_test, y_pred)
print('La matrice de confusion sur le jeu de test :\n', cm_test, '\n')

# On calcul le score d accuracy
acc_train=accuracy_score(y_test, y_pred)
print('L accuracy sur le jeu de test est :\n', acc_train)


Comments : 
- l'accuracy n'est pas tres tres bonne, on remarque sur la matrice de confusion que nos données test ne sont pas tres bien predit ( 143 + 1025 ) sur l'ensemble de test qui sont mal classé . 
- notre model overfitt rapidement, ce qui fait qu'on ne peux pas l'entrainer plus. 

Tous les RNN ont des boucles de rétroaction dans la couche récurrente. Cela leur permet de conserver des informations en « mémoire » au fil du temps. Mais, il peut être difficile de former des RNN standard pour résoudre des problèmes qui nécessitent l'apprentissage de dépendances temporelles à long terme. C'est parce que le gradient de la fonction de perte décroît de façon exponentielle avec le temps  (problème de fuite de gradient)

# RNN-LSTM gauche-droite classique 

In [ ]:
embed_size = 128                                                   # dimension de l'embedding
RNN_size   = 64

# Example d'architecture pour le réseau RNN simple avec dropout
model_lstm = Sequential()
model_lstm.add(Embedding(num_words, embed_size))                 # layer embedding
model_lstm.add(LSTM(RNN_size,return_sequences = False))          # layer lstm
#model_lstm.add(Dropout(0.25))                                    # layer Dropout
model_lstm.add(Dense(1))                                         # layer Dense

# On affiche l'architecture de notre modèle
model_lstm.summary()

# On spécifie la fonction de perte, l'optimiseur, et la fonction d'évaluation
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# On entraine le réseau
batch_size = 32                                                             # tailles des mini-batch
epochs = 10                                                                 # nombre d'époques
history_lstm = model_lstm.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2) # on entraine

In [ ]:
plot_history(history_lstm)

In [ ]:
y_pred=predict(model_lstm)

In [ ]:
# On importe les librairies pour l'évaluation
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

# On calcule la matrice de confusion
cm_test = confusion_matrix(y_test, y_pred)
print('La matrice de confusion sur le jeu de test :\n', cm_test, '\n')

# On calcul le score d accuracy
acc_train=accuracy_score(y_test, y_pred)
print('L accuracy sur le jeu de test est :\n', acc_train)

Comments :
-  On a une accuracy bien meilleur avec le LSTM


- Les réseaux LSTM sont un type de RNN qui utilise des unités spéciales en plus des unités standard. Les unités LSTM comprennent une « cellule mémoire » qui peut conserver des informations en mémoire pendant de longues périodes. Un ensemble de portes est utilisé pour contrôler quand les informations entrent dans la mémoire, quand elles sont sorties et quand elles sont oubliées. Cette architecture leur permet d'apprendre les dépendances à plus long terme. 

# LSTM bi-directionnel et return_sequences = False

In [ ]:
embed_size = 128                                                   # dimension de l'embedding
RNN_size   = 64

# Example d'architecture pour le réseau RNN simple avec dropout
model_bidi = Sequential()
model_bidi.add(Embedding(num_words, embed_size))                 # layer embedding
model_bidi.add(Bidirectional(LSTM(RNN_size, return_sequences = False)))    # layer RNN
#model_bidi.add(Dropout(0.25))                                    # layer Dropout
model_bidi.add(Dense(1))                                         # layer Dense

# On affiche l'architecture de notre modèle
model_bidi.summary()

# On spécifie la fonction de perte, l'optimiseur, et la fonction d'évaluation
model_bidi.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# On entraine le réseau
batch_size = 32                                                            # tailles des mini-batch
epochs = 10                                                                 # nombre d'époques
history_bidi = model_bidi.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2) # on entraine

In [ ]:
plot_history(history_bidi)

In [ ]:
y_pred=predict(model_bidi)

In [ ]:
# On importe les librairies pour l'évaluation
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

# On calcule la matrice de confusion
cm_test = confusion_matrix(y_test, y_pred)
print('La matrice de confusion sur le jeu de test :\n', cm_test, '\n')

# On calcul le score d accuracy
acc_train=accuracy_score(y_test, y_pred)
print('L accuracy sur le jeu de test est :\n', acc_train)

L'accuracy est un peu plus meilleur que pour les LSTM bi

L'utilisation de la bidirectionnelle exécutera nos entrées de deux manières, l'une du passé au futur et l'autre du futur au passé. Ce qui fait la difference avec les LSTM normal qui sont capables à tout moment de préserver les informations du passé et du futur.



# Plusieurs couches LSTM.

In [ ]:
embed_size = 64                                                 # dimension de l'embedding
RNN_size   = 64

model = Sequential()
model.add(Embedding(num_words, embed_size, mask_zero=True))   
model.add(Bidirectional(LSTM(RNN_size,return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))


model.summary()
# On spécifie la fonction de perte, l'optimiseur, et la fonction d'évaluation
model.compile(loss=BinaryCrossentropy(from_logits=True),
              optimizer=Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train, epochs=10,
                    validation_data=(x_test,y_test),
                    validation_steps=30)

In [ ]:
plot_history(history)

In [ ]:
y_pred=predict(model)

In [ ]:
# On importe les librairies pour l'évaluation
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

# On calcule la matrice de confusion
cm_test = confusion_matrix(y_test, y_pred)
print('La matrice de confusion sur le jeu de test :\n', cm_test, '\n')

# On calcul le score d accuracy
acc_train=accuracy_score(y_test, y_pred)
print('L accuracy sur le jeu de test est :\n', acc_train)